# Augmented Dickey-Fuller Test

Since the test-statistic is much greater than the critical values at 1, 5,, or 10% levels, we cannot reject the null hypothesis and therefore we do not have a mean reverting series : this is quite obvious since equity markets in general do not follow a mean reverting process; they perfrom random walks as Geometric Brownian Motions (GBMs)

In [9]:
from __future__ import print_function
import yfinance as yf
import statsmodels.tsa.stattools as ts
from datetime import datetime

# Download Amazon stock data from Yahoo Finance using yfinance
amzn = yf.download("AMZN", start="2022-01-01", end="2023-01-01")

# Perform the Augmented Dickey-Fuller test
result = ts.adfuller(amzn['Adj Close'], 1)

# Print the result
print(result)



[*********************100%%**********************]  1 of 1 completed

(-1.3842480616312725, 0.5898040665184731, 0, 250, {'1%': -3.456780859712, '5%': -2.8731715065600003, '10%': -2.572968544}, 1395.7346239519522)


In [5]:
1- 0.04917754919243083

0.9508224508075692

# Hurst Exponent Test for Stationarity

The goal here is to obtain a scalar value which will help us to identify whether a series is mean-reverting, random walking, or trending

We take the logarithm of the price to stabilize fluctuations, makes the data more normal, and analyze data proportionately

We then find the variance of log price to assess the rate of diffusive behaviour

In Geometric Brownian Motion (GBM), at large time intervals, the variance of the time lag is proportional to the time lag itself ; if this holds, we have a GBM ; if this does not hold , we either have a trending series or a mean-reverting series

If any sequential price movements possess a non-zero correlationship among themselves, then the variance of the lag is proportional to time lag to the power 2*H ; where H is the Hurst Exponent Value

If H < 0.5 : the series is mean-reverting

If H = 0.5 : the series is Geometric Brownian Motion

If H > 0.5 : The series is trending

Smaller the H, more is the extent of the mean-reverting nature of the time series

Greater the H ,more is the extent of the trending nature of the time series



In [20]:
from __future__ import print_function
import yfinance as yf
from datetime import datetime
from numpy import cumsum, log, polyfit, sqrt, std, subtract
from numpy.random import randn
import numpy as np

def hurst(ts):
    lags = range(2, 100)
    tau = []

    # Calculate the lagged variances
    for lag in lags:
        lagged_diff = subtract(ts[lag:], ts[:-lag])
        lagged_diff_std = std(lagged_diff)
        tau.append(sqrt(lagged_diff_std))

    log_lags = log(lags)
    log_tau = log(tau)

    # Debugging: Print the log values
    print("log_lags:", log_lags)
    print("log_tau:", log_tau)

    # Check for NaN or infinite values before polyfit
    if np.isnan(log_lags).any() or np.isnan(log_tau).any():
        print("NaN values found in log_lags or log_tau")
    if np.isinf(log_lags).any() or np.isinf(log_tau).any():
        print("Infinite values found in log_lags or log_tau")

    poly = polyfit(log_lags, log_tau, 1)
    print("poly:", poly)

    return poly[0] * 2.0

# Download Amazon stock data
amzn = yf.download("GOOG", start="2013-01-01", end="2016-12-31")

# Remove NaN values
amzn = amzn.dropna()

# Remove non-positive values
amzn = amzn[amzn['Adj Close'] > 0]

# Generate random data for comparison
gbm = log(cumsum(randn(100000)) + 1000)
mr = log(randn(100000) + 1000)
tr = log(cumsum(randn(100000) + 1) + 1000)

print("Hurst(GBM): %s" % hurst(gbm))
print("Hurst(MR): %s" % hurst(mr))
print("Hurst(TR): %s" % hurst(tr))
print("Hurst(AMZN): %s" % hurst(amzn['Adj Close']))


[*********************100%%**********************]  1 of 1 completed


log_lags: [0.69314718 1.09861229 1.38629436 1.60943791 1.79175947 1.94591015
 2.07944154 2.19722458 2.30258509 2.39789527 2.48490665 2.56494936
 2.63905733 2.7080502  2.77258872 2.83321334 2.89037176 2.94443898
 2.99573227 3.04452244 3.09104245 3.13549422 3.17805383 3.21887582
 3.25809654 3.29583687 3.33220451 3.36729583 3.40119738 3.4339872
 3.4657359  3.49650756 3.52636052 3.55534806 3.58351894 3.61091791
 3.63758616 3.66356165 3.68887945 3.71357207 3.73766962 3.76120012
 3.78418963 3.80666249 3.8286414  3.8501476  3.87120101 3.8918203
 3.91202301 3.93182563 3.95124372 3.97029191 3.98898405 4.00733319
 4.02535169 4.04305127 4.06044301 4.07753744 4.09434456 4.11087386
 4.12713439 4.14313473 4.15888308 4.17438727 4.18965474 4.20469262
 4.21950771 4.2341065  4.24849524 4.26267988 4.27666612 4.29045944
 4.30406509 4.31748811 4.33073334 4.34380542 4.35670883 4.36944785
 4.38202663 4.39444915 4.40671925 4.41884061 4.4308168  4.44265126
 4.4543473  4.46590812 4.47733681 4.48863637 4.4998096

<ipython-input-20-5d487d87d99d>:19: RuntimeWarning: divide by zero encountered in log
  log_tau = log(tau)
